In [ ]:
# Import python packages
import pandas as pd

# Feature store
from snowflake.ml.feature_store import FeatureStore
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Model registry
from snowflake.ml.registry import Registry
registry = Registry(session=session, database_name="dev", schema_name="data_science")

#### Get inference dataset from FS

In [ ]:
# Connect to existing Feature Store
fs = FeatureStore(
    session=session,
    database="dev",
    name="user_activity_feature_store",
    default_warehouse="analytics_wh_flex"
)

In [ ]:
# Retrieve required feature views
fv_ads = fs.get_feature_view(name="user_features_ads", version="1.2")
fv_call_quality = fs.get_feature_view(name="user_features_call_quality", version="1.2")
fv_call_rating = fs.get_feature_view(name="user_features_call_rating", version="1.2")
fv_data_usage = fs.get_feature_view(name="user_features_data_usage", version="1.2")
fv_drawer_event = fs.get_feature_view(name="user_features_drawer_event", version="1.2")
fv_inbound_calls = fs.get_feature_view(name="user_features_inbound_calls", version="1.2")
fv_messages = fs.get_feature_view(name="user_features_messages", version="1.2")
fv_nps_rating = fs.get_feature_view(name="user_features_nps_rating", version="1.2")
fv_outbound_calls = fs.get_feature_view(name="user_features_outbound_calls", version="1.2")
fv_redial = fs.get_feature_view(name="user_features_redial", version="1.2")
fv_sessions = fs.get_feature_view(name="user_features_sessions", version="1.2")

all_features = [
    fv_ads,
    fv_call_quality, 
    fv_call_rating,
    fv_data_usage,
    fv_drawer_event,
    fv_inbound_calls,
    fv_messages,
    fv_nps_rating,
    fv_outbound_calls,
    fv_redial,
    fv_sessions  
]

In [ ]:
# Inference spine df
spine_infere_df= session.sql("""
SELECT
    up.user_id_hex,
    m.date_utc
FROM dev.data_science.metrics_daily_userlevel_app_time_sessions m
JOIN dev.data_science.user_profiles up 
      ON m.username = up.latest_username
WHERE m.date_utc = '2025-11-18'
GROUP BY up.user_id_hex, m.date_utc
ORDER BY up.user_id_hex, m.date_utc
""")
# spine_infere_df.show()

In [ ]:
#Generate training dataset
inference_df = fs.generate_training_set(
    spine_df=spine_infere_df,
    features=all_features,
)

# inference_df.show()

In [ ]:
# to_pandas (limit for notebook demo)
inference_df = inference_df.limit(5000).to_pandas().fillna(0)
print(f"Inference samples: {len(inference_df)}")
inference_df.head()

In [ ]:
# Prepare features for inference
inference_variables = inference_df.drop(['USER_ID_HEX', 'DATE_UTC'], axis=1)
print(f"Features: {inference_variables.shape[1]}")

In [ ]:
# Retrieve the default model version from registry
MODEL_NAME = "XGBOOST_MODEL"
mv = registry.get_model(MODEL_NAME).default
model = mv.load()
print(f"Loaded: {mv.model_name} v{mv.version_name}")

In [ ]:
# Predict
prediction = model.predict(inference_variables)

In [ ]:
# Final results
results_df = inference_df[['USER_ID_HEX', 'DATE_UTC']].copy()
results_df['PREDICTED_ACTIVE_DAYS'] = prediction

print(f"Predictions generated: {len(results_df)}")
print(f"\nPrediction distribution:")
print(results_df['PREDICTED_ACTIVE_DAYS'].describe())

results_df.head(10)